# Imports and functions

In [2]:
from imutils import face_utils
from PIL import Image, ImageDraw
import numpy as np
import imutils
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
import dlib
import cv2
import os, subprocess, csv, glob
import matplotlib.pyplot as plt
import audiolabel
import re
from skimage.draw import polygon
from scipy.ndimage.measurements import center_of_mass
from scipy.ndimage import zoom
% matplotlib inline

vre = re.compile(
         "^(AA|AE|AH|AO|EH|ER|EY|IH|IY|OW|UH|UW|R|L)$"
      )

**Installs**

Several installs are required to use the packages in this notebook. The cv2 package can be installed most easily through conda install: "conda install -c menpo cv3." This is somewhat confusingly called "cv3", apparently to indicate its compatibility with python 3.x, even though it is still imported as cv2.

The dlib install instructions can be found at http://www.pyimagesearch.com/2017/03/27/how-to-install-dlib/ . Imutils is easier to install with pip; simply do "pip install imutils".

These installs would simply be included in the notebook, but doing the dlib installs actually takes quite a while (5-10 minutes) and a fairly large amount of disk space (~1GB).

**Required files**

Beyond the installs, dlib needs a face training data set in order for the landmark predictor to be instantiated successfully. The 68-point frontal face dataset that is standard for use of dlib's landmark predictor can be found at https://github.com/davisking/dlib-models. (This comes as a .bz2 archive and needs to be uncompressed before use.)

In [3]:
# instantiate face detector and landmark predictor (TODO outside of fcn)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

These first two functions are critical for any further processing. get_video_frame grabs a raw video frame from a desired time, while get_norm_video_frame applies a resizing and warping algorithm to the raw frame's face bounding box and returns only that portion. Note that this involves detecting the eye landmarks and using them to determine the angle that the face is off of the horizontal, as well as the relative size of the face or closeness to the camera. The second function is heavily inspired by the demo script at https://www.pyimagesearch.com/2017/05/22/face-alignment-with-opencv-and-python/. 

In [7]:
def get_video_frame(video, time):
    """
    Return the single frame closest to the given timepoint. Can then run detect_landmarks on the frame, ideally
      as an input to get_norm_face.
    Inputs: video - an MXF file; time in seconds - format d.ddd (sec.msec), rounded to three decimal places.
    Outputs: an ndarray image of the desired frame.
    """
    output_bmp = 'temp.bmp'
    try:
        os.remove(output_bmp)
    except OSError:
        pass
    frame_get_args = ['ffmpeg', '-i', video, 
                      '-vcodec', 'bmp', 
                      '-ss', time,
                      '-vframes', '1', 
                      '-an', '-f', 'rawvideo',
                       output_bmp]
    subprocess.check_call(frame_get_args)
    frame = cv2.imread(output_bmp)
    return frame

In [4]:
def detect_landmarks(my_ndarray, detector=detector, predictor=predictor):
    """
    Inputs: an ndarray frame output from cv2.VideoCapture object, 
            a detector of choice from dlib,
            and a dlib face landmark predictor trained on data of choice.
    Output: a (68,2) ndarray containing X,Y coordinates for the 68 face points dlib detects.
    """

    # read in image TODO change to something more general like the commented-out line
    gray = cv2.cvtColor(my_ndarray, cv2.COLOR_BGR2GRAY)
    # gray = np.asarray(cv2image, dtype=np.uint8)
    
    # TODO cheekpad obliteration happens here if remove_cheekpad=True
    
    # run face detector to get bounding rectangle
    rect = detector(gray, 1)[0]
    
    # run landmark prediction on portion of image in face rectangle; output
    shape = predictor(gray, rect)
    shape_np = face_utils.shape_to_np(shape)
    
    return shape_np

In [ ]:
def get_norm_face(video, time, detector, predictor):
    """
    Return a affine-transformed and rescaled face bounding box on the basis of the distance between the eye landmarks
      and their angle relative to each other. Transformation centers the (pictured person's) left eye in a fixed location
      in an ndarray of a fixed size and warps/resizes the image such that the right eye's center forms a horizontal line
      with the left eye, and is a fixed distance away from it.
    """
    # initialize util
    detector = detector
    predictor = predictor
    fa = FaceAligner(predictor, desiredFaceWidth = 256) # TODO adjust as needed
    
    # get the raw frame
    frame = get_video_frame(video, time)
    frame = imutils.resize(frame, width=800) # TODO adjust as needed
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rect = detector(gray,1)
    
    # ... TODO get this to work together
    
    # run imutils.resize on rect
    #(x, y, w, h) = rect_to_bb(rect)
    faceAligned = fa.align(frame, gray, rect)
    
    return faceAligned

The next function is heavily inspired by: http://codegists.com/snippet/python/facial_landmark_generatorpy_habanoz_python

In [5]:
def draw_landmarks(my_ndarray, shape, aperture_xy = False):
    """
    Inputs: an ndarray frame output from a cv2.VideoCapture object, and a (68,2) ndarray of x,y coords that dlib detects.
    Outputs: an image with lines drawn over the detected landmarks; useful for testing and visualization.
    aperture_xy: if True, also draw (next to face) numerical values for x and y diameters of lip aperture.
    """

    out_image = my_ndarray.copy()

    for i,name in enumerate(face_utils.FACIAL_LANDMARKS_IDXS.keys()):
        if name == "mouth":
            continue
        j,k = face_utils.FACIAL_LANDMARKS_IDXS[name]
        pts = np.array(shape[j:k], dtype=np.uint32)
        for idx,pt in enumerate(pts):
            pt1 = pt
            try:
                pt2 = pts[idx+1]
            except IndexError:
                if name == "left_eye" or name == "right_eye":
                    pt2 = pts[0]
                else:
                    continue
            cv2.line(out_image, tuple(pt1), tuple(pt2), (255,255,255))
    
    # drawing the mouth with some more precision
    # draw most of the outer perimeter of lips
    jm,km = face_utils.FACIAL_LANDMARKS_IDXS['mouth']
    for idx in range(jm,jm+11): 
        pt1 = shape[idx]
        pt2 = shape[idx+1]
        cv2.line(out_image, tuple(pt1), tuple(pt2), (255,255,255))
    
    # draw the last segment for the outer perimiter of lips
    cv2.line(out_image, tuple(shape[48]), tuple(shape[59]), (255,255,255))
    
    # draw the inner aperture of the lips
    for idx in range(jm+12,km):
        pt1 = shape[idx]
        try:
            pt2 = shape[idx+1]
        except IndexError:
            pt2 = shape[jm+12]
        cv2.line(out_image, tuple(pt1), tuple(pt2), (255,255,255))
        
    # add text indicating measured lip aperture in px
    if aperture_xy:
        x,y = get_lip_aperture(shape)
        add_string = "x={}, y={}".format(round(x,1),round(y,1))
        loc = tuple(np.subtract(shape[4], (200,0)))
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(out_image, add_string, loc, font, 0.8, (255,255,255), 2, cv2.LINE_AA)
        
    return out_image

In [6]:
def get_lip_aperture(shape):
    """
    Inputs: the typical 68,2 ndarray shape object output by detect_landmarks.
    Outputs: a 2-tuple of horizontal and vertical diameters of the lip aperture,
     treating the horizontal line like the major axis of an ellipse,
     and the vertical line like the minor axis.
    """
    horizontal_axis = np.linalg.norm(shape[60] - shape[64])
    vertical_axis = np.linalg.norm(shape[62] - shape[66])
    
    return horizontal_axis,vertical_axis

In [108]:
def lip_mask(frame, shape):
    """
    Returns a simplified ndarray containing 0s/1s, with lips a filled polygon of 1s
    """
    # fetch indices of mouth landmark points
    jm,km = face_utils.FACIAL_LANDMARKS_IDXS['mouth']
    
    # initialize a blank background in the shape of the original image
    mask_dims = [frame.shape[0],frame.shape[1]]
    mask = np.zeros(ground_dims, dtype=np.uint8)

    # fill outer lip polygon
    mouth_outer = shape[jm:jm+11]
    mouth_outer_col = [p[0] for p in mouth_outer]
    mouth_outer_row = [p[1] for p in mouth_outer]
    # last point's coords need to be appended manually
    mouth_outer_col.append(shape[59][0])
    mouth_outer_row.append(shape[59][1])
    rr,cc = polygon(mouth_outer_row,mouth_outer_col)
    mask[rr,cc] = 1

    # then, cancel out inner polygon
    mouth_inner = shape[jm+12:km]
    mouth_inner_col = [p[0] for p in mouth_inner]
    mouth_inner_row = [p[1] for p in mouth_inner]
    rr,cc = polygon(mouth_inner_row,mouth_inner_col)
    mask[rr,cc] = 0

    return mask

The functions in the two cells below are largely a product of Michelle Ching (with some modifications from me).

In [109]:
def get_center(image):
    """ Returns the center location of an array as (x, y). """
    return np.array([image.shape[0] // 2, image.shape[1] // 2])
  
def centroid(image):
    """ Returns centroid. """
    if (image == 0).all():
        return get_center(image)
    centroid = center_of_mass(image)
    centroid = np.rint(np.array(centroid))
    return centroid.astype(int)

def cross(image, point):
    """ Point-marking (with cross shape) function, to mark off i.e. centroid. """
    image_w, image_h = image.shape
    x, y = point
    blank = np.zeros(image.shape)
    blank[max(0, x - 3):min(x + 4, image_w), y] = 1
    blank[x, max(0, y - 3):min(y + 4, image_h)] = 1
    blank[x,y] = 0
    return blank.astype(bool).astype(int)

In [ ]:
def crop_center(input_mask):
    """ 
    Returns mask (1s/0s) with centroid of 1s centered on center of a smaller ground of 0s. 
    This normalizes for some aspects of head movement (relative size and absolute position),
      but does not normalize for left/right head tilt.
    """
    mask = input_mask.copy()
    # define a smaller ground of zeros
    # TODO make this scaled to the individual acquisition somehow, by e.g. lip width
    ground_shape = np.array([int(d/6) for d in mask.shape])
    ground = np.zeros(ground_shape)
    
    # get mask's centroid and ground's center point
    cg_x, cg_y = get_center(ground)
    mask_centroid = centroid(mask)
    ci_x, ci_y = mask_centroid
    # unpack dims of mask and ground
    image_w, image_h = mask.shape
    ground_w, ground_h = ground_shape
    
    # determine preliminary amounts to crop from left and top
    left_diff = ci_x - cg_x
    top_diff = ci_y - cg_y
    
    # check if left/top sides of image fit on ground; chop off if not
    if left_diff > 0:
        mask = mask[left_diff:,]
        ci_x -= left_diff
    if top_diff > 0:
        mask = mask[:,top_diff:]
        ci_y -= top_diff
    
    # get mask dims again
    image_w, image_h = mask.shape
        
    # get right and bottom dimensions
    ground_r = ground_w - cg_x
    ground_b = ground_h - cg_y
    image_r = image_w - ci_x
    image_b = image_h - ci_y 
    
    # determine amounts to crop from right and bottom
    right_diff = ground_r - image_r
    bottom_diff = ground_b - image_b
    
    # check if right/bottom sides of image fit on ground; chop off if not
    if right_diff < 0:
        mask = mask[:image_w + right_diff]
    if bottom_diff < 0:
        mask = mask[:,:image_h + bottom_diff]
        
    image_w, image_h = mask.shape

    # copy modified mask onto ground shape
    left_start = cg_x - ci_x
    top_start = cg_y - ci_y
    ground[left_start:left_start + image_w, top_start:top_start + image_h] += mask
        
    return ground

# Demos

This cell will extract and display a single landmark-annotated video frame.

In [ ]:
video = 'SN12_AA032001.MXF'
time = '00:00:10.900'

frame = get_video_frame(video,time)
shape = detect_landmarks(frame)
color_corrected = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
plt.imshow(draw_landmarks(color_corrected,shape))

This (slightly messy) cell extracts a frame corresponding to the desired timepoint of an audio file (here, from a TextGrid) and extracts a polygon of the lip shape.

In [ ]:
babb = 'vowels/P1.MXF'
babb_tg = os.path.splitext(babb)[0] + ".TextGrid"
pm = audiolabel.LabelManager(from_file=babb_tg, from_type="praat")
v = pm.tier('phone').search(vre)[0] # should only be one match per file; TODO check while running
midpoint= v.center
ffmpeg_time = str(round(midpoint,3))
frame = get_video_frame(babb,ffmpeg_time)
shape = detect_landmarks(frame)
color_corrected = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
plt.imshow(draw_landmarks(color_corrected,shape))

image = lip_mask(frame,shape)
plt.imshow(crop_center(image))

This cell clips a short stretch of video from the longer videos typical of ultrasound acquisitions.

In [ ]:
exp_movie = '12-video/SN12_AA032001.MXF'
exp_snippet = '12-video/SN12_AA032001_short.MXF'
fname = os.path.split(exp_movie)[1]
basename = os.path.splitext(fname)[0]

# get a snippet of the MXF
snippet_args = ['ffmpeg', '-ss', '00:00:30', 
                '-i', exp_movie, 
                "-t", "00:00:08", 
                "-vcodec", "copy", 
                "-acodec", "copy", 
                exp_snippet]
subprocess.check_call(snippet_args)

And this cell takes a short clip and creates (and saves) a GIF with the facial landmarks drawn over the subject's face.

In [ ]:
exp_movie = 'vowels/POO1.MXF'
fname = os.path.split(exp_movie)[1]
basename = os.path.splitext(fname)[0]

subprocess.check_call(['ffmpeg','-loglevel','8','-i',exp_snippet,'-vf','fps=20','img_%05d_f.bmp'])
movie = cv2.VideoCapture('img_%05d_f.bmp')

frame_num = 1

while(movie.isOpened()):
    ret, frame = movie.read()
    if (ret==False):   # the file is finished
        break

    # detect face region and draw landmarks on the image.
    shape = detect_landmarks(frame)
    out_image = draw_landmarks(frame,shape)
    
    # write the image to a .bmp file, with zero-padding to ensure the frames are input 
    cv2.imwrite('{0:05d}g.bmp'.format(frame_num), out_image)
    frame_num += 1
        
# cleanup
cv2.destroyAllWindows()

# make a gif (a bit slowed down by default) from the output bmps
subprocess.check_call(['convert', '*g.bmp', basename+'.gif'])

# remove the input bmps
for bmp in glob.glob("*g.bmp"):
    os.remove(bmp)
for bmp in glob.glob("*f.bmp"):
    os.remove(bmp)

# Development area

In [88]:
# YET TO DO
# tilt normalization
# get all lip shape outputs and store in ndarray
# for each lip shape object (slice of ndarray):
#     get centroid/mean
#     move these to sufficiently large ground (on subject-to-subject basis) centered on ground